In [1]:
# Importing necessary libraries for the model prediction.
import numpy as np
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from random import choice
import vlc
import time
from time import process_time
from playsound import playsound

In [2]:
# Load the trained model
model = keras.models.load_model(r"C:\Users\Manan Patel\Data_Mining Project\Finalized Data\MobileNet_ft.h5")
background = None
accumulated_weight = 0.5

# Assign the co-ordinates for the ROI
ROI_top = 0
ROI_bottom = 300
ROI_left = 0
ROI_right = 320

In [3]:
def cal_accum_avg(frame, accumulated_weight):
    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    
    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY_INV)
    
    # Fetching contours in the frame (These contours can be of hand or any other object in foreground)...
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # If length of contour list = 0, means we didn't get any contours...
    if len(contours) == 0:
        return None
    else:
        # The largest external contour should be the hand...
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        # Returning the hand segment (Max contour) and the threshold image of hand...
        
        return (thresholded, hand_segment_max_cont)

In [5]:
cam = cv2.VideoCapture(0)
num_frames =0
i =0
word_dict = {0:'Paper', 1:'Rock', 2:'Scissor', 3:'none'}  # Using dictionary, Label the predicted output

# Computer move ROI
x_left = 320
y_top = 0
x_right = x_left + 320
y_down = y_top + 300

# Playing back ground music
player = vlc.MediaPlayer("mario.mp3")


Prev_move = None
Computer_move = "none"
winner = "Waiting..."

# Define a fuction to calculate the winner for the game
def calculate_winner(move1, move2):
    if move1 == move2:
        return "Tie"
    if move1 == "Rock":
        if move2 == "Scissor":
            return "Winner"
        if move2 == "Paper":
            return "Looser"
    if move1 == "Scissor":
        if move2 == "Paper":
            return "Winner"
        if move2 == "Rock":
            return "Looser"
    if move1 == "Paper":
        if move2 == "Rock":
            return "Winner"
        if move2 == "Scissor":
            return "Looser"

# We are using the time counter to minimize the false changes in the game.\
# i.e. Sudden change in the action will not consider
start = process_time()        
count = 0
count1 = 0


while True:

    ret, frame = cam.read()
    
    # Fliping the video frame.
    frame = cv2.flip(frame, 1)
    frame_copy = frame.copy()

    # Defining the ROI co-ordinates.
    roi = frame[ROI_top:ROI_bottom, ROI_left:ROI_right]
    
    # Converting the RGB frame to gray
    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    
    # Printing the GUI for the game and resizing the shape.
    score = cv2.imread('scoreboard.jpg') 
    score = cv2.resize(score, (640, 180))
    frame_copy[300:480,0:640] = score
    
    # First few frames, fetching the background...
    if num_frames < 90:
        
        cal_accum_avg(gray_frame, accumulated_weight)
        
        cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT", (80, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
        if num_frames <30:
            cv2.putText(frame_copy, "3...", (135, 230), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (115,95,237), 4)
            cv2.putText(frame_copy, "3...", (440, 230), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (194,129,81), 4)
        elif num_frames <60 and num_frames>29:
            cv2.putText(frame_copy, "2...", (135, 230), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (115,95,237), 4)
            cv2.putText(frame_copy, "2...", (440, 230), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (194,129,81), 4)
        elif num_frames <90 and num_frames>59:
            cv2.putText(frame_copy, "1...", (135, 230), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (115,95,237), 4)
            cv2.putText(frame_copy, "1...", (440, 230), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (194,129,81), 4)
    
    elif num_frames <120 and num_frames>89:
        cv2.putText(frame_copy, "Go", (115, 230), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (115,95,237), 4)
        cv2.putText(frame_copy, "Go", (420, 230), cv2.FONT_HERSHEY_SIMPLEX, 3.0, (194,129,81), 4)
    
    # After that, fetching the object.
    else: 
        hand = segment_hand(gray_frame)
        
        
        if hand is not None:
            
            thresholded, hand_segment = hand
            
            # Drawing the countour around the object...
            cv2.drawContours(frame_copy, [hand_segment + (ROI_left, ROI_top)], -1, (255, 0, 0),1)
            cv2.imshow("Thesholded Hand Image", thresholded)
            
            
            # Resizing the ROI image.
            thresholded = cv2.resize(thresholded, (224, 224))
            thresholded = cv2.cvtColor(thresholded, cv2.COLOR_GRAY2RGB)
            thresholded = np.reshape(thresholded, (1,thresholded.shape[0],thresholded.shape[1],3))
            
            # Predicting the model output and assign it as user move.
            pred = model.predict(thresholded)
            User_move = word_dict[np.argmax(pred)]

            # Logical conditions...
            if Prev_move != User_move:
                u = 1
                end = process_time()
                total = (end-start)*100
                if User_move != "none" and total > 300:  # Total time create a constraint to minimize the false move.
                    
                    start = 0
                    end = 0
                    Computer_move = choice(['Rock', 'Paper', 'Scissor']) # Random choice generation by computer.
                    winner = calculate_winner(User_move, Computer_move)
                    start = process_time()
                    if winner == "Winner":
                        count = count + 1  # Counting the winner's move
                    if winner == "Looser":
                        count1 = count1 + 1  # Counting the Looser's move
                        
                if User_move == "none":  # When User is not ready to play
                    Computer_move = "none"
                    winner = "Waiting..."
            
            
            if Computer_move != "none":  # Getting computer move as image

                icon = cv2.imread(Computer_move + '.' + 'jpg')
                icon = cv2.resize(icon, (320, 300))
                frame_copy[y_top:y_down,x_left:x_right] = icon
        
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame_copy, "Your Move: " + User_move, (105, 350), font, 0.5, (115,95,237), 2, cv2.LINE_AA)
            cv2.putText(frame_copy, "Computer's Move: " + Computer_move, (350, 350), font, 0.5, (194, 129, 81), 2, cv2.LINE_AA)
            cv2.putText(frame_copy, winner, (280, 450), font, 0.5, (255,255,255), 2, cv2.LINE_AA)
            
            cv2.putText(frame_copy, str(count), (183, 420), font, 0.7, (115,95,237), 2, cv2.LINE_AA)
            cv2.putText(frame_copy, str(count1), (438, 420), font, 0.7, (194, 129, 81), 2, cv2.LINE_AA)

            Prev_move = User_move
                    
      
    cv2.rectangle(frame_copy, (ROI_right, ROI_top), (ROI_left, ROI_bottom), (194,129,81), 3)
    cv2.rectangle(frame_copy, (x_right, y_top), (x_left, y_down), (194,129,81), 3)
    num_frames += 1

    
#     cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
#     cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow("Rock, Paper, Scissor...", frame_copy)
    player.play()  # Play the music
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        player.stop()
        break
        
# Release the camera and destroy all the windows

cam.release()
cv2.destroyAllWindows()